## Génération de données aléatoires pour renseigner la table personnes

In [1]:
from pyspark.sql import SparkSession
from faker import Faker
import pandas as pd
import random



### Demarrage de la session Spark. Si Spark pushdown active, verifier dans HUE qu'un Job Yarn est créé

In [9]:
%%time
# MASTER="yarn"         # Spark distant sur CDP  (Spark pushdown)
MASTER="local[*]"       # Spark local sur CML

spark = SparkSession.builder.appName("0_ingestion.ipynb").master(MASTER).getOrCreate()

CPU times: user 28.9 ms, sys: 24.9 ms, total: 53.9 ms
Wall time: 485 ms


In [10]:
# Test de la connexion Spark
spark.sql("show databases").show()

Hive Session ID = 5a80a6b4-9b72-45f4-98fb-554b94bb59b2


+------------------+
|      databaseName|
+------------------+
|           default|
|information_schema|
|        jmontenaro|
|           olivier|
|               sys|
+------------------+



### Paramètres d'initialisation de la génération de données

In [11]:
LIGNES=20000      # Nombre de lignes de la dataframe Spark de destination
PAS=1500          # Nombre de lignes de la dataframe Pandas intermédiaire

### Génération de données aléatoire à partir d'un Faker Python

In [12]:
%%time

fake = Faker()             # Initialisation du Faker

df = pd.DataFrame()        # Création de la dataframe Pandas pour les données du Faker

i = 0
while i < LIGNES:
          
    i = i + 1

    data = {
        'id': fake.random_number(digits=5),
        'age': random.randint(18, 70),
        'sexe': random.choice(['homme', 'femme']),
        'niveau_education': random.choice(['CAP/BEP', 'BACCALAUREAT', 'BTS/DUT',  'LICENSE', 'MAITRISE', 'MASTER', 'DOCTORAT']),
        'duree_chomage': random.randint(0, 24),
        'experience_ant': random.randint(0, 20),
        'competence': fake.job(),
        'formation': random.choice([True, False]),
        'taux_chomage_local': random.uniform(2, 20),
        'secteur_activite': fake.company_suffix(),
        'reseau_professionnel': random.randint(0, 100),
        'support_social': random.randint(0, 10),
        'retour_emploi': random.choice([1, 0])
        }
    
    df = pd.concat( [df, pd.DataFrame(data, index=['id'])] )
    
    if (i%PAS == 0) : # A chaque remplissage de la dataframe pandas, chargement en bulk de la dataframe spark

       sdf = spark.createDataFrame(df) if (i == PAS) else sdf.unionAll(spark.createDataFrame(df))
         
       df  = df.drop(df.index)
    
       print ("Iteration: ", i/PAS)
        


    
    

Iteration:  1.0
Iteration:  2.0
Iteration:  3.0
Iteration:  4.0
Iteration:  5.0
Iteration:  6.0
Iteration:  7.0
Iteration:  8.0
Iteration:  9.0
Iteration:  10.0
Iteration:  11.0
Iteration:  12.0
Iteration:  13.0
CPU times: user 43.8 s, sys: 5.07 s, total: 48.9 s
Wall time: 53 s


In [13]:
sdf.count()

19500

In [ ]:
TABLE="olivier.personnes"

sdf.write.format("parquet").mode("overwrite").saveAsTable(TABLE)


In [ ]:
sdf.describe()

In [16]:
spark.stop()